In [160]:
import pandas as pd
import numpy as np

In [301]:
def process_activity_column(column):
    change_str = ""
    changing = False
    processed_column = column.copy()
    max_stack = [0]
    max_stack_index = []
    for index, state in enumerate(column):
        if str(state).split()[-1] == 'begin':
            changing = True
#             print("Begin Found")
            if not change_str == '':# and (state.strip().split()[0] not in change_str.split('-')):
                change_str += '-'+ state.strip().split()[0]
            else:
                change_str = state.strip().split()[0]
#             print(index, change_str)
        if changing:
            processed_column[index] = change_str.split('-')
            if len(processed_column[index]) > max_stack[0]:
                max_stack = [len(processed_column[index])]
                max_stack_index = [index]
            elif len(processed_column[index]) == max_stack[0]:
                max_stack.append(len(processed_column[index]))
                max_stack_index.append(index)
        if str(state).split()[-1] == 'end':
            processed_column[index] = change_str.split('-')
#             print("End Found")
            end_act = state.strip().split()[0]
#             print("End act: ", end_act)
            acts = change_str.split('-')
            acts.remove(end_act)
            if not acts:
#                 print("no acts")
                changing = False
            change_str = '-'.join(acts)
#             print(index, acts, change_str)
    return processed_column, max_stack, max_stack_index

In [302]:
data_milan = pd.read_csv("dataset/milan/data_test", sep = "\t", index_col=0, parse_dates=[0], infer_datetime_format=True, skip_blank_lines = True, names = ['Timestamp (UTC)', 'sensorID', 'state', 'activity'])
data_milan = data_milan.sort_index()
print(data_milan.tail(10))

                           sensorID state              activity
Timestamp (UTC)                                                
2009-10-16 15:52:22.000088     M012    ON                   NaN
2009-10-16 15:52:25.000010     M027    ON  Kitchen_Activity end
2009-10-16 15:52:27.000033     M027   OFF                   NaN
2009-10-16 15:52:30.000057     M012   OFF                   NaN
2009-10-16 15:52:31.000004     M027    ON                   NaN
2009-10-16 15:52:31.000084     M003    ON                   NaN
2009-10-16 15:52:32.000064     M010    ON                   NaN
2009-10-16 15:52:34.000046     M027   OFF                   NaN
2009-10-16 15:52:34.000091     M011    ON                   NaN
2009-10-16 15:52:36.000072     M003   OFF                   NaN


In [303]:
processed_column, max_stack, max_stack_index = process_activity_column(list(data_milan['activity']))
data_milan['mod_col'] = processed_column
data_milan['mod_col'] = data_milan['mod_col'].fillna('None')

In [304]:
print(max_stack, max_stack_index)
print(data_milan.iloc[2661])
print(data_milan.index[0] + pd.Timedelta(minutes=15))

[2, 2, 2, 2, 2, 2] [2661, 2662, 2663, 2664, 2665, 2666]
sensorID                              M001
state                                   ON
activity                  Leave_Home begin
mod_col     [Kitchen_Activity, Leave_Home]
Name: 2009-10-16 14:49:54.000046, dtype: object
2009-10-16 00:16:04.000059


In [387]:
def process_subset(df, uni_list_beg, cols):
    act_col = df['mod_col'].tolist()
    uni_list = []
    for i in act_col:
        uni_list += ['Other'] if i == 'None' else i
    uni_list = set(uni_list)
    act_col_beg_end = df['activity'].tolist()
    for index, state in enumerate(act_col_beg_end):
        if str(state).split()[-1] == 'begin':
            uni_list_beg.append(state.strip().split()[0])
        elif str(state).split()[-1] == 'end' and (str(state).split()[0] in uni_list_beg):
            uni_list_beg.remove(str(state).split()[0])
    uni_list = list(uni_list)
    uni_list.sort()
    sensorID = df['sensorID'].tolist()
    sensor_list = [0] * (len(cols)-1)
    for index, sensor in enumerate(cols[:-1]):
        sensor_list[index] = int(sensor in sensorID)
    return uni_list, uni_list_beg, sensor_list

In [484]:
cols = data_milan.sensorID.unique()
#print(list(cols))
cols = list(cols)
cols.append('Activities')
df_sensors = pd.DataFrame(columns = cols)
df_sensors.head(5)

uni_list_beg = []
startTime = data_milan.index[0]
stepTime = pd.Timedelta(minutes=1)
endTime = data_milan.index[-1]
count = 0
countc = 0
while startTime  <= endTime:
    df_15mins = data_milan.loc[startTime: startTime + stepTime]
    #print(df_15mins)
    startTime = startTime + stepTime
#     print(startTime)
    uni_list, uni_list_beg, sensor_list =  process_subset(df_15mins, uni_list_beg, cols)
    print("count: ", count)
    print("List of open activities:", uni_list_beg)
    print("List of activities:", uni_list)
    #print(list(cols))
    count+=1
    print("Sensor Feature:", sensor_list)
    if (all(v == 0 for v in sensor_list) and uni_list_beg) or uni_list:
        print("countc: ", countc)
        countc+=1
        if all(v == 0 for v in sensor_list):
            uni_list = list(uni_list_beg)
        list2append = uni_list
        sensor_list.append(list2append)
        sensor_series = pd.Series(sensor_list, index= df_sensors.columns)      
        print(list(sensor_series))
        df_sensors = df_sensors.append(sensor_series, ignore_index=True)

count:  0
List of open activities: []
List of activities: ['Other']
Sensor Feature: [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  0
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Other']]
count:  1
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  2
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  3
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  4
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
coun

count:  81
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  82
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  83
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  84
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  85
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  86
List of open activities: []
List of activities: ['Other']
Sensor Feature: [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 

count:  137
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  138
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  139
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  140
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  141
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  142
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

count:  205
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  206
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  207
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  208
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  209
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
count:  210
List of open activities: []
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

count:  247
List of open activities: ['Sleep']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  30
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Sleep']]
count:  248
List of open activities: ['Sleep']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  31
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Sleep']]
count:  249
List of open activities: ['Sleep']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  32
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Sleep']]
count:  250
List of open activities: ['Sleep']
List of activities: []
Sensor Feature: [0, 0

...

count:  770
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  521
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  771
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  522
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  772
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  523
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  773
List of open activities: ['Leave_Home']
List o

count:  809
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  560
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  810
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  561
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  811
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  562
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  812
List of open activities: ['Leave_Home']
List o

count:  853
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  604
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  854
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  605
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  855
List of open activities: ['Leave_Home']
List of activities: []
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
countc:  606
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['Leave_Home']]
count:  856
List of open activities: ['Leave_Home']
List o

count:  896
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  647
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  897
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  648
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  898
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  649
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

count:  940
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
countc:  691
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  941
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
countc:  692
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ['Kitchen_Activity']]
count:  942
List of open activities: ['Kitchen_Activity']
List of activities: ['Kitchen_Activity']
Sensor Feature: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
countc:  693
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,

In [485]:
pd.set_option('display.max_rows', df_sensors.shape[0]+1)
df_sensors

,M017,M009,M019,M020,T001,T002,M016,M015,M022,M012,...,M018,M007,M027,M002,M001,D001,M004,D003,D002,Activities
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
5,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
6,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
7,0,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,[Other]
8,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,[Other]
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,[Other]


In [486]:
df_sensors.to_csv('dataset/milan/data_processed_test.csv')#, index = False)

In [487]:
df_unique_rows = df_sensors.iloc[:,:-1]

In [488]:
df_unique_rows = df_unique_rows.drop_duplicates()

In [489]:
df_unique_rows

,M017,M009,M019,M020,T001,T002,M016,M015,M022,M012,...,M026,M018,M007,M027,M002,M001,D001,M004,D003,D002
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
`